#Imports

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving pacientes.csv to pacientes (1).csv
Saving experimento_teste_ab.csv to experimento_teste_ab (1).csv


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm
from scipy.stats import chi2_contingency
from scipy.stats import shapiro

In [ ]:
df_pacientes = pd.read_csv('pacientes.csv')
df_ab = pd.read_csv('experimento_teste_ab.csv')

In [ ]:
df_pacientes.head()

,Unnamed: 0,Pressao_Arterial,Idade,Genero,Etnia,Estado_Saude,Nome_Genero,Nome_Etnia,Nome_Estado_Saude
0,0,142.755809,31,1,1,1,Feminino,Afro-americano,Com condições de saúde adicionais
1,1,161.878468,35,1,1,0,Feminino,Afro-americano,Saudável
2,2,161.508259,39,0,3,1,Masculino,Asiático,Com condições de saúde adicionais
3,3,144.979408,43,1,3,1,Feminino,Asiático,Com condições de saúde adicionais
4,4,162.502954,55,0,2,1,Masculino,Hispânico,Com condições de saúde adicionais


In [ ]:
df_ab.head()

,Visitante_ID,Versão_Página,Visualizações,Conversões
0,1,A,167,1
1,2,A,160,0
2,3,A,170,0
3,4,A,161,1
4,5,A,104,0


#1

Qual dos cenários tem a maior taxa de conversão?

In [ ]:
taxas_conversao = df_ab.groupby('Versão_Página').agg(
    taxa_conversao=('Conversões', 'sum'),
    visualizacoes=('Visualizações', 'sum')
)

taxas_conversao['taxa_conversao'] = taxas_conversao['taxa_conversao'] / taxas_conversao['visualizacoes']

print(taxas_conversao[['taxa_conversao']])

               taxa_conversao
Versão_Página                
A                    0.000696
B                    0.000830


#2

Calcule qual o tamanho da amostra necessária para o desenvolvimento
de um teste A/B, seguindo os seguintes critérios:

a. O cenário A, da base, como o inicial, que funciona hoje.

b. Considere 95% de confiança de que o efeito na conversão não foi
aleatório com um nível de signicância de 5% (alpha = 0,05).

c. Também considere 80% de certeza conseguir capturar o efeito
da nova abordagem.

d. O aumento para 10% de conversão.

In [ ]:
p0 = 0.2  # Taxa de conversão do cenário A (20%)
p1 = 0.3  # Taxa de conversão do cenário B (30%)
alpha = 0.05  # Nível de significância
power = 0.8  # Poder do teste (80%)

sample_size = sm.stats.NormalIndPower().solve_power(effect_size=sm.stats.proportion_effectsize(p0, p1), alpha=alpha, power=power, ratio=1)

print(f"Tamanho da amostra necessário: {round(sample_size)}")

Tamanho da amostra necessário: 292


#3

Considerando uma amostra de 45 números que representam o index do
dataframe, índices= ([909, 751, 402, 400, 726, 39, 184, 269, 255, 769,
209, 715, 677, 381, 793, 697, 89, 280, 232, 756, 358, 36, 439, 768, 967,
699, 473, 222, 89, 639, 883, 558, 757, 84, 907, 895, 217, 224, 311, 348,
146, 505, 273, 957, 362]). Considerando essa amostra é possível dizer
que a idade média das pessoas com problemas cardíacos é maior que
50 anos? Nível de signicância igual a 5%.

In [ ]:
indices = [909, 751, 402, 400, 726, 39, 184, 269, 255,
           769, 209, 715, 677, 381, 793, 697, 89, 280,
           232, 756, 358, 36, 439, 768, 967, 699, 473,
           222, 89, 639, 883, 558, 757, 84, 907, 895,
           217, 224, 311, 348, 146, 505, 273, 957, 362]

amostra_idades = df_pacientes.loc[indices, 'Idade']

media_populacional = 50
nivel_significancia = 0.05

media_amostra = amostra_idades.mean()
desvio_padrao_amostra = amostra_idades.std()
tamanho_amostra = len(amostra_idades)

t_estatistica = (media_amostra - media_populacional) / (desvio_padrao_amostra / np.sqrt(tamanho_amostra))
graus_liberdade = tamanho_amostra - 1

valor_critico = stats.t.ppf(1 - nivel_significancia, graus_liberdade)

rejeitar_h0 = t_estatistica > valor_critico

print(f'Média da amostra: {media_amostra}')
print(f'Desvio padrão da amostra: {desvio_padrao_amostra}')
print(f'Estatística t: {t_estatistica}')
print(f'Valor crítico t: {valor_critico}')
print(f'Rejeitar H₀? {"Sim" if rejeitar_h0 else "Não"}')

Média da amostra: 50.08888888888889
Desvio padrão da amostra: 11.40538593309443
Estatística t: 0.052280983519350496
Valor crítico t: 1.6802299765721167
Rejeitar H₀? Não


#4

Queremos entender que tipo de amostra estamos lidando se dividirmos
os conjuntos em 2, sendo um com pessoas que têm condições de
saúde adicionais e o outro com pessoas saudáveis. Seria dependente
ou independente?

In [ ]:
print(df_pacientes['Estado_Saude'].value_counts())

amostra_com_condicoes = df_pacientes[df_pacientes['Estado_Saude'] == 1]
amostra_saudaveis = df_pacientes[df_pacientes['Estado_Saude'] == 0]

print(f"Quantidade de pessoas com condições de saúde adicionais: {len(amostra_com_condicoes)}")
print(f"Quantidade de pessoas saudáveis: {len(amostra_saudaveis)}")

print("As amostras são independentes porque estamos lidando com dois grupos distintos (doentes e saudáveis).")

Estado_Saude
1    510
0    490
Name: count, dtype: int64
Quantidade de pessoas com condições de saúde adicionais: 510
Quantidade de pessoas saudáveis: 490
As amostras são independentes porque estamos lidando com dois grupos distintos (doentes e saudáveis).


#5

Agora considere o um conjunto de pessoas aleatória que representam
o index do dataframe, índices = ([690, 894, 67, 201, 364, 19, 60, 319,
588, 643, 855, 623, 530, 174, 105, 693, 6, 462, 973, 607, 811, 346, 354,
966, 943, 372]), podemos dizer que a pressão arterial média para
pacientes com condições de saúde adicionais é igual à pressão arterial
média para pacientes sem condições adicionais de saúde? Considere o
nível de signicância a 6%.

In [ ]:
indices = [690, 894, 67, 201, 364, 19,
           60, 319, 588, 643, 855, 623,
           530, 174, 105, 693, 6, 462,
           973, 607, 811, 346, 354, 966,
           943, 372]

amostra_selecionada = df_pacientes.iloc[indices]

amostra_com_condicoes = amostra_selecionada[amostra_selecionada['Estado_Saude'] == 1]
amostra_saudaveis = amostra_selecionada[amostra_selecionada['Estado_Saude'] == 0]

media_com_condicoes = amostra_com_condicoes['Pressao_Arterial'].mean()
media_saudaveis = amostra_saudaveis['Pressao_Arterial'].mean()

desvio_com_condicoes = amostra_com_condicoes['Pressao_Arterial'].std()
desvio_saudaveis = amostra_saudaveis['Pressao_Arterial'].std()

tamanho_com_condicoes = len(amostra_com_condicoes)
tamanho_saudaveis = len(amostra_saudaveis)

#Teste T

numerador = abs(media_com_condicoes - media_saudaveis)
denominador = np.sqrt((desvio_com_condicoes**2 / tamanho_com_condicoes) + (desvio_saudaveis**2 / tamanho_saudaveis))

t_estatistica = numerador / denominador

graus_liberdade = min(tamanho_com_condicoes -1, tamanho_saudaveis -1)

nivel_significancia = 0.06

valor_critico = stats.t.ppf(1 - 0.03, graus_liberdade)

rejeitar_h0 = abs(t_estatistica) > valor_critico

print(f'Média da pressão arterial para pessoas com condições de saúde adicionais: {media_com_condicoes}')
print(f'Média da pressão arterial para pessoas saudáveis: {media_saudaveis}')
print(f'Estatística t calculada: {t_estatistica}')
print(f'Valor crítico t: {valor_critico}')
print(f'Rejeitar H₀? {"Sim" if rejeitar_h0 else "Não"}')

Média da pressão arterial para pessoas com condições de saúde adicionais: 153.19725449805927
Média da pressão arterial para pessoas saudáveis: 152.94305165531088
Estatística t calculada: 0.08304391716460978
Valor crítico t: 2.120233533791387
Rejeitar H₀? Não


#6

Existe uma diferença signicativa na pressão arterial média entre
diferentes grupos étnicos nesta população? (Teste ANOVA, alpha é 5%)

a. Hipótese Nula (H0): A pressão arterial média é a mesma em todos
os grupos étnicos.

b. Hipótese Alternativa (H1): Há uma diferença signicativa na
pressão arterial média entre pelo menos dois grupos étnicos
nesta população.

In [ ]:
grupos_etnia = [df_pacientes[df_pacientes['Etnia'] == etnia]['Pressao_Arterial']
                for etnia in df_pacientes['Etnia'].unique()]

f_estatistica, p_value = stats.f_oneway(*grupos_etnia)

alpha = 0.05

print(f'Estatística F: {f_estatistica}')
print(f'Valor p: {p_value}')

if p_value < alpha:
    print('Rejeitamos a hipótese nula. Há uma diferença significativa entre as médias.')
else:
    print('Não rejeitamos a hipótese nula. Não há diferença significativa entre as médias.')

Estatística F: 0.23245296261596837
Valor p: 0.873800123850745
Não rejeitamos a hipótese nula. Não há diferença significativa entre as médias.


#7

Lúcia é uma pesquisadora e tem o objetivo de entender a relação de
gênero neste grupo de pacientes. Acredita-se que há uma relação
entre o sexo e condições de saúde adicionais. (Teste qui-quadrado)

In [ ]:
tabela_contingencia = pd.crosstab(df_pacientes['Genero'], df_pacientes['Estado_Saude'])

chi2_stat, p_value, dof, expected = chi2_contingency(tabela_contingencia)

alpha = 0.05

print(f'Estatística qui-quadrado: {chi2_stat}')
print(f'Valor p: {p_value}')
print(f'Graus de liberdade: {dof}')
print(f'Tabela esperada:\n{expected}')

if p_value < alpha:
  print('Rejeitamos a hipótese nula. Existe uma relação significativa entre sexo e condições de saúde.')
else:
  print('Não rejeitamos a hipótese nula. Não existe uma relação significativa entre sexo e condições de saúde.')

Estatística qui-quadrado: 0.0
Valor p: 1.0
Graus de liberdade: 1
Tabela esperada:
[[243.53 253.47]
 [246.47 256.53]]
Não rejeitamos a hipótese nula. Não existe uma relação significativa entre sexo e condições de saúde.


#8

Existe uma associação entre a idade dos pacientes e sua pressão
arterial?

a. Hipótese Nula (H0): se a pressão arterial é independente da idade

b. Hipótese Alternativa (H1): a pressão arterial esta associada a
idade

In [ ]:
correlacao, p_value = stats.pearsonr(df_pacientes['Idade'], df_pacientes['Pressao_Arterial'])

alpha = 0.05

print(f'Coeficiente de correlação de Pearson: {correlacao}')
print(f'Valor p: {p_value}')

if p_value < alpha:
    print('Rejeitamos a hipótese nula. Existe uma associação significativa entre a idade e a pressão arterial.')
else:
    print('Não rejeitamos a hipótese nula. Não existe uma associação significativa entre a idade e a pressão arterial.')

Coeficiente de correlação de Pearson: -0.02138535693464897
Valor p: 0.49936082120556174
Não rejeitamos a hipótese nula. Não existe uma associação significativa entre a idade e a pressão arterial.


#9

Qual é o intervalo de confiança para a média da pressão arterial entre
os pacientes com condições de saúde adicionais? (nível de conança
95%)

In [ ]:
amostra_com_condicoes = df_pacientes[df_pacientes['Estado_Saude'] == 1]['Pressao_Arterial']

media = amostra_com_condicoes.mean()
desvio_padrao = amostra_com_condicoes.std()
tamanho_amostra = len(amostra_com_condicoes)

nivel_confianca = 0.95
t_critico = stats.t.ppf(1 - (1 - nivel_confianca) / 2, df=tamanho_amostra-1)

erro_padrao = desvio_padrao / (tamanho_amostra ** 0.5)

margem_erro = t_critico * erro_padrao
intervalo_inferior = media - margem_erro
intervalo_superior = media + margem_erro

print(f'Intervalo de confiança para a média da pressão arterial (95%): ({intervalo_inferior}, {intervalo_superior})')

Intervalo de confiança para a média da pressão arterial (95%): (149.21972443653837, 151.01717945377672)


#10

A distribuição da pressão arterial na população segue uma distribuição
normal?

a. Hipótese Nula (H0): A distribuição da pressão arterial na
população segue uma distribuição normal

b. Hipótese Alternativa (H1): A distribuição da pressão arterial na
população não segue uma distribuição normal

In [ ]:
stat, p_value = shapiro(df_pacientes['Pressao_Arterial'])

print(f'Estatística de teste: {stat}')
print(f'Valor p: {p_value}')

alpha = 0.05
if p_value < alpha:
    print('Rejeitamos a hipótese nula. A distribuição da pressão arterial não segue uma distribuição normal.')
else:
    print('Não rejeitamos a hipótese nula. A distribuição da pressão arterial segue uma distribuição normal.')

Estatística de teste: 0.9970366758795889
Valor p: 0.060729578232028174
Não rejeitamos a hipótese nula. A distribuição da pressão arterial segue uma distribuição normal.
